In [1]:
import os
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import numpy
import math
from time import time

In [2]:
def PRINCIPAL(INPUTPATH, OUTPUTPATH, T, K, H, tecnica, tipoDistancia):
    #LLamamos a la función CalcularFotogramasClave para obtener la lista
    #de fotogramas solución                 
    ListaKeyFrames, ListaFramesBuenos = CalcularFotogramasClave(INPUTPATH, T, K, H, tecnica, tipoDistancia)
    
    
    #Sacamos la lista de imagenes del INPUTPATH
    ListaImagenes=[]
    indice=0
    
    imagenes=os.listdir(INPUTPATH)
    
    while indice<len(imagenes):
        input_path=os.path.join(INPUTPATH,imagenes[indice])
        img = cv2.imread(input_path)
        ListaImagenes.append(img)
        indice=indice+T
    
    #Luego sacamos las que queremos(De los indices de ListaKeyFrames)
    imagenesSolucion=[]
    for i in range(K):
        #Sacamos por cada frame solcion de ListaKeyFrames su posicion en la ListaFramesBuenos
        # e incluimos en imagenesSolucion la imagen con dicha posición en ListaFrames(Donde están
        #todas las imagenes del INPUTPATH)
        imagenesSolucion.append(ListaImagenes[ListaFramesBuenos.index(ListaKeyFrames[i])])
        
    #Escribimos dichas imágenes en la carpeta OUTPATH
    indiceimagen=1
    for img in imagenesSolucion:
        s= OUTPUTPATH +'\imagen'+repr(indiceimagen)+'.jpg'
        cv2.imwrite(s,img)
        indiceimagen+=1
    
    tiempo_final = time() #Guardamos el tiempo final antes de crear el video
    
    #Por último llamamos a la función creaVideo
    creaVideo(imagenesSolucion,OUTPUTPATH)
    
    return tiempo_final
    
    

In [3]:
def CalcularFotogramasClave(INPUTPATH, T, K, H, tecnica, tipoDistancia):
    #extraemos las imágenes de la ruta especificada
    ListaFrames=[]
    indice=0
    
    imagenes=os.listdir(INPUTPATH)
    
    
    #Para cada fotograma en INPUTPATH, indice=o, indice<=N, indice+=T
    while indice<len(imagenes):
        input_path=os.path.join(INPUTPATH,imagenes[indice])
        img = cv2.imread(input_path)
        
        #Añadir a datos el conjunto de histogramas RGB (de tamaño H) del fotograma f
        
        if tecnica is 'media': #comprueba si queremos hacer la media a los datos
            ListaFrames.append(media_histogramas(img, H))
           
        
        elif tecnica is 'concatena': #comprueba si queremos concatenar los datos
            ListaFrames.append(concatena_histogramas(img, H))
        
        indice=indice+T

    
    #Llamamos a una función que aplica el algoritmo y devuelve los centroides
    ListaCentroides = aplicaKMedias(ListaFrames, K)
    
    #Calculamos cuales son aquellos fotogramas del INPUTPATH más cercanos a los centroides
    #a partir de un tipoDistancia, esos fotogramas serán la solución
    ListaKeyFrames = CalculaCentroidesClases(ListaCentroides, ListaFrames, K, tipoDistancia)
    
    return ListaKeyFrames,ListaFrames
    

In [4]:
#FUNCIONES DE TÉCNICAS DE AGRUPAR INFORMACIÓN DE HISTOGRAMAS

#Media de los 3 histogramas(RGB)
def media_histogramas(img, H):
    color = {'b','g','r'}
    
    sumaHistogramas=[] #Creamos lista vacia donde se va a acumular la suma de los 3 histogramas
    
    for i in range(H):#Llenamos esa lista con H ceros, para luego ir sumando los valores a esos ceros
        sumaHistogramas.append(0)
    
        
    for i,col in enumerate(color):#Sacamos los 3 histogramas de cada foto
        histr = cv2.calcHist([img],[i],None,[H],[0,256])
        
        for posicion in range(H):#Recorremos cada histograma y lo vamos acumulando en SumaHistogramas
            sumaHistogramas[posicion] += histr[posicion][0]
        
    
    for valor in range(H):#Dividimos entre 3 para hacer la media de los 3 histogramas
        sumaHistogramas[valor] = sumaHistogramas[valor]/3
            
    return sumaHistogramas

#Concatenación de los 3 histogramas(RGB)
def concatena_histogramas(img, H):
    color = {'b', 'g', 'r'}
    
    listaConcatenada=[] #Creamos la lista donde vamos a concatenar los 3 arrays del histograma de cada color RGB
    
    for i,col in enumerate(color):#Sacamos los 3 histogramas de cada foto y para cada histogramas hacemos lo siguiente:
        histr = cv2.calcHist([img],[i],None,[H],[0,256])
        
        
        for posicion in histr: #recorremos todo el histograma
            listaConcatenada.append(posicion[0]) #vamos añadiendo a la lista todos los valores del histograma
    
    return listaConcatenada

In [5]:
#FUNCIÓN PARA APLICAR EL ALGORITMO K-MEDIAS
def aplicaKMedias(ListaFrames, K):
    
    #ajustamos los datos con número de clusters K y hacemos la llamada al algoritmo ya implmentado por sklearn
    kmeans = KMeans(n_clusters=K).fit(ListaFrames)

    #calculamos los centroides de los clústeres
    listaCentroides = kmeans.cluster_centers_
    
    return listaCentroides

In [6]:
#FUNCIÓN PARA SACAR LOS FRAMES MAS CERCANOS A LOS CENTROIDES
def CalculaCentroidesClases(ListaCentroides, ListaFramesBuenos, K, tipoDistancia):
    ListaKeyFrames = [] #inicializamos la lista de fotogramas clave
    
    #dependiendo del valor del parámetro tipoDistancia haremos el cálculo de los fotogramas cercanos con un método u otro
    if tipoDistancia is 'euclidea': 
        for k in range(K): #para cada centroide k 
            fotograma_cercano_k = distancia_euclidea(ListaCentroides, ListaFramesBuenos, k) #fotograma cercano a k
            ListaKeyFrames.append(fotograma_cercano_k)
    
    elif tipoDistancia is 'hamming':
        for k in range(K):
            fotograma_cercano_k = distancia_hamming(ListaCentroides, ListaFramesBuenos, k)
            ListaKeyFrames.append(fotograma_cercano_k)
    
    elif tipoDistancia is 'manhattan':
        for k in range(K):
            fotograma_cercano_k = distancia_manhattan(ListaCentroides, ListaFramesBuenos, k)
            ListaKeyFrames.append(fotograma_cercano_k)
    
    return ListaKeyFrames

In [7]:
#FUNCIONES PARA CALCULAR LAS DISTANCIAS

#Distancia Hamming
def distancia_hamming(listaCentroides,listaDatosBuenos,indiceCentroide):
    res=0                                              #inicializamos a 0
    minimo=-1                                          #inicializamos el minimo
    
    
    #para cada fotograma comparamos con cuantos valores del histograma dista del centroide
   
    for posibleCentro in listaDatosBuenos:             #para cada fotograma (posible centro)
        acumulador=0                                   #creamos el acumulador 
        for i in range(0,len(posibleCentro)):          #para cada valor del fotograma
            if posibleCentro[i]!=listaCentroides[indiceCentroide][i]: #si el valor es distinto del valor del centro en la misma posicion
                acumulador=acumulador+1                #incrementamos a 1
        if (minimo== -1 or acumulador<minimo):         #una vez terminamos de acumular, comprobamos si el acumuladorBase 
                                                       #es igual a -1 o si los valores distantes del fotograma con el centro 
                                                       #es menor que el ya teníamos
            
            res=posibleCentro                          #si esto es asi pasa a ser el nuevo posible centro más cercano
            minimo=acumulador                          #el nuevo valor a comparar será el acumulador del nuevo posible centro
    
    return res

#Distancia Manhattan
def distancia_manhattan(listaCentroides,listaDatosBuenos,indiceCentroide):
    res=0  #inicializamos a 0
    minimo=-1 #inicializamos el minimo
    
    
    for posibleCentro in listaDatosBuenos:                   #para cada fotograma (posible centro)
        acumulador=0 #creamos el acumulador
        for i in range(0,len(posibleCentro)):                #para cada valor del fotograma
            
            #Calculamos SUMATORIO(|posibleCentro[i]-listaCentroides[indiceCentroide][i])
            resta=posibleCentro[i]-listaCentroides[indiceCentroide][i] 
            if resta<0:
                resta=resta*-1
            acumulador=acumulador+resta                      #lo sumamos al acumulador
    
        if (minimo== -1 or acumulador<minimo):               #si la distancia es menor que la que ya teníamos
            res=posibleCentro                                #sustituimos el nuevo posible centro
            minimo=acumulador                                #sustituimos el nuevo valor a comparar
    
    return res

#Distancia Euclidea
def distancia_euclidea(listaCentroides, listaDatosBuenos, indiceCentroide):
        res = 0                                                              #inicializamos a 0
        minimo = -1                                                          #inicializamos el minimo
            
        for posibleCentro in listaDatosBuenos:                               #para cada fotograma (posible centro)
            operacion = 0                                                    #creamos el acumulador
            for i in range(0, len(posibleCentro)):                           #para cada valor del fotograma
                operacion += (posibleCentro[i] - listaCentroides[indiceCentroide][i])**2 #calculamos: (xi -yi)^2
                
            valorRaiz = math.sqrt(operacion)                                 #le hacemos la raiz
            
            if(minimo == -1 or valorRaiz<minimo):                            #si la distancia es menor que la que ya teníamos
                minimo = valorRaiz                                           #sustituimos el nuevo valor a comparar
                res = posibleCentro                                          #sustituimos el nuevo posible centro
                    
        return res

In [8]:
#FUNCIÓN PARA CREAR UN VIDEO A PARTIR DE UNA IMAGENES
def creaVideo(imagenes,OUTPUTPATH):
    img1 = imagenes[0] #sacamos una imagen cualquiera, por ejemplo la 0
    height , width , layers=  img1.shape #usamos la medidas de dicha imagen
    video = cv2.VideoWriter(OUTPUTPATH +'\solucionVideo.avi',-1,1,(width,height)) #creamos el video con las medidas de la imagen
    
    for img in imagenes:       #para cada imagen
        for numero in range(3):
            video.write(img)
        
    cv2.destroyAllWindows()
    video.release()

In [16]:
inputpath="D:\Archivos\Archivos\Desktop\FramesVideo1\Video1"
outputpath="D:\Archivos\Archivos\Desktop\GuardaImagen"

tiempo_inicial = time()
tiempo_final=PRINCIPAL(inputpath, outputpath, 40, 20, 256, 'media', 'euclidea')
tiempo_ejecucion=tiempo_final-tiempo_inicial
print('El tiempo de ejecución es:',tiempo_ejecucion)


El tiempo de ejecución es: 0.9557290077209473
